<a href="https://colab.research.google.com/github/unverciftci/JAX-Denemeleri/blob/master/JAX_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google JAX İle Hızlandırılmış Hesaplama ve Otomatik Türev Alma 

Not: Uygulamamızda GPU kullanacağız. Runtime kısmından "Change runtime type", oradan da GPU'yu seçin. 

JAX aslında NumPy kütüphanesinin GPU ve TPU gibi hızlandırıcılara
ve otomatik türev almaya uygun olarak geliştirilmişidir diyebiliriz. 

Otomatik türev alma ([Auotograd](https://github.com/HIPS/autograd) veya [Autodiff](https://jax.readthedocs.io/en/latest/notebooks/autodiff_cookbook.html)) sayesinde döngülerin, koşulların (if gibi) ve türevin türevinin türevi bile alınabilir.    

JAX, [XLA](https://github.com/HIPS/autograd) derleyicisini (compiler) kullanarak kodu GPU ve TPU hızlandırıcılarında çalıştırır. Derleme, gerekli kütüphanelerin çağrılması ile hali hazırda çalışır. Fakat siz de kendi fonksiyonlarınızı jit (just-in-time) derleyicisine tanıtabilirsiniz.

Derleme ve otomatik türev birleştirilebilir ve karmaşık algoritmalarda bile azami performans sağlanır.

Şimdi gerekli kütüphaneleri çağıralım.

In [0]:
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random

Matrislerin Çarpımı

Birazdan rastgele veri üreteceğiz. NumPy ile Jax rastegele sayıları farklı şekilde üretmektedir. Bizim için bunun önemi paralel hesaplamada ortaya çıkıyor. Ayrıntılar için JAX dökümantosyonuna bakın. 

Rastgele sayı oluşturmak için bir anahtar (key) tanımlıyoruz. Buradan yarı rastgele sayı üretebiliriz (PRNG). 

In [0]:
key = random.PRNGKey(0)

Yukarıda bir şekilde rastgele sayıların her defasında aynı çıkması için kök (seed) oluşturduk (parantez içindeki 0 yerine başka sayı alarak bu değiştirilebilir).

Mesela bileşenleri normal dağılımdan oluşturulmuş 10 boyutlu vektör olan x:

In [4]:
x = random.normal(key, (10,))
print(x)

[-0.3721109   0.26423115 -0.18252768 -0.7368197  -0.44030377 -0.1521442
 -0.67135346 -0.5908641   0.73168886  0.5673026 ]


Büyük matrisleri çarpalım.

In [5]:
size = 3000
x = random.normal(key, (size, size), dtype=np.float32)
%time y = np.dot(x, x.T)
%time y = np.dot(x, x.T).block_until_ready()

CPU times: user 443 ms, sys: 318 ms, total: 762 ms
Wall time: 1.98 s
CPU times: user 11.7 ms, sys: 8.31 ms, total: 20 ms
Wall time: 20.1 ms


`block_until_ready` dememizin sebebi şu: JAX normalde işlemi sonlandırmıyor sadece hazır hale getiriyor, eğer çıktı isterseniz veya başka yerde kullanılacaksa işleme devam ediyor. Bu şekilde işlemin tamamının süresin öğrendik.



İsterseniz NumPy ile karşılaştıralım.

In [6]:
import numpy as onp
%time y = onp.dot(x, x.T)

CPU times: user 1.55 s, sys: 42.6 ms, total: 1.6 s
Wall time: 854 ms


Daha yavaş oldunu gördünüz. Bunu da hızlanırmanın bir yolu var:

In [9]:
from jax import device_put

x = random.normal(key, (size, size), dtype=np.float32)
x = device_put(x)
%time y = np.dot(x, x.T).block_until_ready()

CPU times: user 10.5 ms, sys: 8.88 ms, total: 19.4 ms
Wall time: 21 ms


Burada `device_put` fonksiyonu `jit(lambda x: x)` görevi yapıyor.

Ama seçenek olarak GPU ya da TPU seçilirse zaten potansiyel kodun işlenmesi daha hızlı olur.

JAX sadece hızlandırılmış NumPy değildir. Program dönüştürücüleri sayısal kod yazarken yararlıdır. Bunların bir kaçı:
*  `jit`, kodu hızlandırır
*  `grad`, türev alır
*  `vmap`, otomatik vektörleştime ya da toplu işlem.

Şimdi bunları inceleyelim.

## Fonksiyonları `jit` ile Hızlandırma 

JAX normalde işlemleri GPU veya TPU'ya birer birer birer gönderiyor. Eğer ardı ardına işlemler yapacaksanız, bunları kapsayan fonksiyonu `@jit` ile kaplarız.

Mesela yapay sinir ağlarında kullanılan `selu` fonksiyonunu tanımlayalım.

In [11]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * np.where(x > 0, x, alpha * np.exp(x) - alpha)

x = random.normal(key, (1000000, ))
%time y = selu(x).block_until_ready()

CPU times: user 72.1 ms, sys: 46.8 ms, total: 119 ms
Wall time: 362 ms


Şimdi `jit` ile hızlandıralım. İlk çağrıldığında derlenecek (just-in-time olarak) diğer seferlerde hazır kullanılacak.

---



In [12]:
selu_jit = jit(selu)
%time selu_jit(x).block_until_ready()

CPU times: user 50.5 ms, sys: 10 ms, total: 60.5 ms
Wall time: 116 ms


DeviceArray([ 2.093448  ,  0.21820937, -0.5104705 , ...,  0.03640566,
              0.7458341 ,  0.20638663], dtype=float32)

## `grad` ile Türev Alma

Sayısal değerler alan fonksiyonların türevlerini almak için `grad` kullanılır.

In [25]:
def sum_logistic(x):
  return np.sum(1.0 / (1 + np.exp(-x)))

x = 3.
derivative_fn = grad(sum_logistic)
der = derivative_fn(x)
print(der)

0.045176663


`grad` ve `jit` bir arada kullanılabilir.

In [26]:
der = grad(jit(sum_logistic))(x)
print(der)

0.045176663


Ya da arka arkaya türev alabiliriz.

In [28]:
der = grad(jit(grad(jit(sum_logistic))))(1.)
print(der)

-0.09085776


## `vmap` ile Otomatik Olarak Vektörleştirme

Döngülerin kodu yavaşlattığını bilirsiniz. `vmap` ile vektörleştirme yapıyoruz ve paralel hesaplama yapılabiliyor.

In [0]:
mat = random.normal(key, (150, 100))
bathced_x = random.normal(key,(10, 100))

Yukarıda `mat` matris, `batched_x` ise yan yana vektörlerden oluşan matristir, tıpkı veri matrisi gibi. Şimdi matrisi her bir vektörle döngü ile çarpalım.

In [44]:
def naive_batched(v_batched):
  return np.stack([np.dot(mat, v) for v in v_batched])

%time z = naive_batched(bathced_x)

CPU times: user 7.01 ms, sys: 0 ns, total: 7.01 ms
Wall time: 7.43 ms


`vmap` ile hızlandıralım.

In [46]:
@jit
def batched(v_batched):
  return np.dot(bathced_x, mat.T)

%time z = batched(bathced_x)

CPU times: user 2.27 ms, sys: 0 ns, total: 2.27 ms
Wall time: 1.57 ms
